<a href="https://colab.research.google.com/github/sahyoun99/rag-cag-implementation/blob/main/Rag_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **install langchain and OpenAI**

*   LangChain is a framework for building applications powered by Large  Language Models (LLMs) like GPT, Claude, etc
*  LangChain helps you connect an AI model to tools, data, memory, and workflows so it can do real tasks—not just chat



In [1]:
pip install -q langchain openai

# Load pdf from drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%pip install -qU langchain-community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [22]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

# Replace '/content/drive/MyDrive/MyDocuments/your_document.pdf' with the actual path to your PDF file in Google Drive
pdf_path = '/content/drive/MyDrive/Real Estate/Real Estate Legislation.pdf' # <--- Please update this path!

loader = PyPDFLoader(pdf_path)
documents = loader.load()

print(f"Successfully loaded {len(documents)} pages from the PDF.")
# You can inspect the first document's content if you wish:
# print(documents[0].page_content[:500])

Streaming output truncated to the last 5000 lines.


Successfully loaded 127 pages from the PDF.


In [9]:
print(f"Successfully loaded {len(documents)} pages from the PDF.")
documents[7]


Successfully loaded 127 pages from the PDF.


Document(metadata={'producer': 'Mac OS X 10.13.6 Quartz PDFContext', 'creator': 'Adobe InDesign CC 2015 (Macintosh)', 'creationdate': '2020-04-09T08:24:20+00:00', 'moddate': '2022-03-11T09:08:31+04:00', 'trapped': '/False', 'source': '/content/drive/MyDrive/Real Estate/Real Estate Legislation.pdf', 'total_pages': 127, 'page': 7, 'page_label': '8'}, page_content='1312\nOccupant   : Any person using a Unit for its intended purposes,\n      including an Owner, a tenant, or any other person\n      authorised by the Owner to use the Unit.\nManagement Company  : A sole proprietorship or a company recognised by\n      RERA and specialised in managing Common\n      Facilities, Jointly Owned Real Property, or Common\n      Parts, as the case may be.\nHotel Project Management Company: A sole proprietorship or a company recognised by\n          RERA and specialised in managing Hotel Projects\n          and their Common Parts in accordance with the\n          provisions of this Law.\nManagement En

# Split the text

In [10]:
pip install -U langchain-text-splitters

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, #size of each chunk
    chunk_overlap = 200 # overlap between chunks
)
# Split the documents into chunks
texts = text_splitter.split_documents(documents)

print(f"Successfully split {len(documents)} pages into {len(texts)} chunks.")


Successfully split 127 pages into 429 chunks.


# Set up OpenAI API Key

In [13]:
pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 3.2 MB/s eta 0:00:00


In [85]:
import os
from google.colab import userdata

# Set the OPENAI_API_KEY environment variable from Colab secrets
# Make sure you have added your OpenAI API key to Colab secrets under the name 'OPENAI_API_KEY'
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

print("OPENAI_API_KEY environment variable set.")

OPENAI_API_KEY environment variable set.


In [87]:
from langchain_openai import OpenAIEmbeddings

# Initialize the OpenAI Embeddings model. It will automatically use the OPENAI_API_KEY from the environment variable.
embeddings = OpenAIEmbeddings()

# Create embeddings for your text chunks
# For large documents, consider processing in batches if memory becomes an issue.
embedded_texts = embeddings.embed_documents([t.page_content for t in texts])

print(f"Successfully created embeddings for {len(embedded_texts)} text chunks.")

Successfully created embeddings for 429 text chunks.


# Create and Save FAISS Vector Store

In [88]:
pip install -qU langchain-community faiss-cpu

In [89]:
from langchain_community.vectorstores import FAISS

# Create a FAISS vector store from the documents and the OpenAI embeddings model
vector_store = FAISS.from_documents(texts, embeddings)

# Save the vector store to a local directory named 'faiss_index'
vector_store.save_local("faiss_index")


vector_store = FAISS.load_local(
    "faiss_index",
    embeddings,
    allow_dangerous_deserialization=True
)

# Create LLM

In [90]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)


# Create retriever

In [91]:
retriever = vector_store.as_retriever(
    search_kwargs={"k": 4}  # top 4 relevant chunks
)


# Ask Questions using the Retrieval Chain

In [92]:
pip install -U langchain langchain-core langchain-community

In [93]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Prompt
prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:
<context>
{context}
</context>

Question: {input}
"""
)

Retrieval chain

In [94]:
from langchain_core.runnables import RunnablePassthrough

retrieval_chain = (
    {
        "context": retriever,
        "input": RunnablePassthrough()
    }
    | prompt
    | llm
)


User chat

In [95]:
print("📘 Document Q&A Chat")
print("Type your question or type 'exit' to quit.\n")

while True:
    user_input = input("You: ").strip()

    if user_input.lower() in {"exit", "quit", "q"}:
        print("👋 Goodbye!")
        break

    if not user_input:
        continue

    response = retrieval_chain.invoke(user_input)

    print("\n🤖 Answer:")
    print(response.content)
    print("-" * 60)


📘 Document Q&A Chat
Type your question or type 'exit' to quit.

You: How does the DLD verify the value of a real property?

🤖 Answer:
The DLD verifies the value of a real property by requiring a Firm to conclude a Real Property Valuation agreement with the entity requesting the valuation. This agreement must outline the rights and obligations of the parties involved and specifically state the real property to be valuated, the purpose of the valuation, and the period for completing the valuation.
------------------------------------------------------------
You: exit
👋 Goodbye!
